In [1]:
import pandas as pd
from scipy.stats import mode

# Clean Answer

In [2]:
# Load the data
data = pd.read_csv('tweets/tweets_filtered.csv')
data = data.iloc[0:5000, :]
data.to_csv('tweets/tweets_filtered_top_5000.csv', index=False)

#### Gpt Informado

In [3]:
folder = 'raw_answers/'
file = 'gpt/gpt_informado.csv'

df = pd.read_csv(folder + file)

# Get only int ids
df = df[df['ID'].apply(lambda x: x.isnumeric())]

# Get first answer from each ID
df = df.groupby('ID').first().reset_index()


df['ID'] = df['ID'].astype(int)

df = df.sort_values('ID').reset_index(drop=True)

df['Sports Betting'] = df['Sports Betting'].apply(lambda x: 1 if 'yes' in x else 0)

df = df.rename(columns={'Sports Betting': 'gpt_informado'})

df.to_csv('clean_answers/gpt/gpt_informado.csv', index=False)

#### Gpt Não-Informado

In [4]:
folder = 'raw_answers/'
file = 'gpt/gpt_nao_informado.csv'

df = pd.read_csv(folder + file)

df['ID'] = df['ID'].astype(str)

# Get only int ids
df = df[df['ID'].apply(lambda x: x.isnumeric())]

df['Mentions Sports Betting'] = df['Mentions Sports Betting'].apply(lambda x: 1 if 'yes' in x else 0)

df = df.rename(columns={'Mentions Sports Betting': 'gpt_nao_informado'})

df = df.drop("Tweet Content", axis=1)

df.to_csv('clean_answers/gpt/gpt_nao_informado.csv', index=False)


#### Llama 2 Informado

In [5]:
folder = 'raw_answers/'
file = 'llama/classificacoes_informadas_llama3.txt'

# Read every line and store in a list
with open(folder + file, 'r') as f:
    lines = f.readlines()

# Store in dataframe with id 1 to 5000
df = pd.DataFrame({'ID': range(1, 5001), 'classificacao': lines})

# Convert all to lower
df['classificacao'] = df['classificacao'].apply(lambda x: x.lower())

# If it has a yes in the string, it is a 1, otherwise 0
df['classificacao'] = df['classificacao'].apply(lambda x: 1 if 'yes' in x else 0)

df = df.rename(columns={'classificacao': 'llama_informado'})

df.to_csv('clean_answers/llama/classificacoes_informadas_llama3.csv', index=False)


#### Llama 2 Não-Informado

In [6]:
folder = 'raw_answers/'
file = 'llama2/classificacoes_informadas.txt'

# Read every line and store in a list
with open(folder + file, 'r') as f:
    lines = f.readlines()

# Store in dataframe with id 1 to 5000
df = pd.DataFrame({'ID': range(1, 5001), 'classificacao': lines})

# Convert all to lower
df['classificacao'] = df['classificacao'].apply(lambda x: x.lower())

# If it has a yes in the string, it is a 1, otherwise 0
df['classificacao'] = df['classificacao'].apply(lambda x: 1 if 'yes' in x else 0)

df = df.rename(columns={'classificacao': 'llama_nao_informado'})

df.to_csv('clean_answers/llama/classificacoes_nao_informadas_llama3.csv', index=False)


# Join Answers

In [7]:
def read_csv_files(csv_files):
    dfs = [pd.read_csv('clean_answers/'+ file) for file in csv_files]
    combined_df = pd.concat(dfs, axis=1, join='inner')
    return combined_df.drop('ID', axis=1)

# Lista de arquivos CSV
folder = 'clean_answers/'
csv_files = ['gpt/gpt_informado.csv', 'gpt/gpt_nao_informado.csv', 'llama/classificacoes_informadas_llama3.csv', 'llama/classificacoes_nao_informadas_llama3.csv']

final_df = read_csv_files(csv_files)

# Get stats from each model

In [8]:
sample = pd.read_csv('tweets/base_to_be_sampled.csv').sample(100, random_state=42)

In [9]:
tweets = pd.read_csv('tweets/tweets_filtered.csv')

In [10]:
tweets_samples = tweets.loc[sample.index]
tweets_samples.to_csv('tweets/tweets_samples.csv', index=False)

In [11]:
human_sample = pd.read_csv('tweets/human_classified_tweets.csv')

In [12]:
human_sample.index = human_sample['ID'] - 1

In [13]:
final_df = pd.concat([final_df, human_sample.drop('ID', axis=1)], axis=1)
final_df = final_df.dropna()
final_df = final_df.rename(columns={'Tweet Content': 'human'})

In [14]:
def evaluate_column_vs_human(final_df, column_to_compare, human_column='human'):
    """
    Compares a specified column with the human-evaluated column and calculates precision, recall, and F1-score.
    """
    TP = final_df[(final_df[column_to_compare] == 1) & (final_df[human_column] == 1)].shape[0]
    FP = final_df[(final_df[column_to_compare] == 0) & (final_df[human_column] == 1)].shape[0]
    FN = final_df[(final_df[column_to_compare] == 1) & (final_df[human_column] == 0)].shape[0]
    TN = final_df[(final_df[column_to_compare] == 0) & (final_df[human_column] == 0)].shape[0]

    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return {
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }



#### Gpt Informado

In [15]:
results = evaluate_column_vs_human(final_df, column_to_compare='gpt_informado')
print(results)


{'Precision': 0.16666666666666666, 'Recall': 0.25, 'F1-Score': 0.2}


#### Gpt Não-Informado

In [16]:
results = evaluate_column_vs_human(final_df, column_to_compare='gpt_nao_informado')
print(results)


{'Precision': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-Score': 0.3333333333333333}


#### Llama Informado

In [17]:
results = evaluate_column_vs_human(final_df, column_to_compare='llama_informado')
print(results)


{'Precision': 0.6666666666666666, 'Recall': 0.08333333333333333, 'F1-Score': 0.14814814814814814}


#### Llama Não-Informado

In [18]:
results = evaluate_column_vs_human(final_df, column_to_compare='llama_nao_informado')
print(results)


{'Precision': 0.16666666666666666, 'Recall': 0.06666666666666667, 'F1-Score': 0.09523809523809522}
